In [1]:
from qick import *

import matplotlib.pyplot as plt
import numpy as np
import time

# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger
import qick.tprocv2_assembler as qasm
from importlib import reload  # Python 3.4+


##### Load FGPA BitStream
soc = QickSoc('./qick_216_spin_qtt.bit')

print(soc)
print(soc.qtt)


ZCU216
ZCU216
QICK running on ZCU216, software version 0.2.273

Firmware configuration (built Fri Aug  2 13:27:25 2024):

	Global clocks (MHz): tProcessor 291.840, RF reference 245.760

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (14.035 us)
		fs=4669.440 MHz, fabric=291.840 MHz, 32-bit DDS, range=4669.440 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (14.035 us)
		fs=4669.440 MHz, fabric=291.840 MHz, 32-bit DDS, range=4669.440 MHz
		DAC tile 1, blk 0 is 0_229, on JHC1

	0 readout channels:

	16 digital output pins:
	0:	PMOD0_0
	1:	PMOD0_1
	2:	PMOD0_2
	3:	PMOD0_3
	4:	PMOD0_4
	5:	PMOD0_5
	6:	PMOD0_6
	7:	PMOD0_7
	8:	PMOD1_0
	9:	PMOD1_1
	10:	PMOD1_2
	11:	PMOD1_3
	12:	PMOD1_4
	13:	PMOD1_5
	14:	PMOD1_6
	15:	PMOD1_7

	tProc qick_processor ("v2") rev 21: program memory 1024 words, data memory 1024 words
		external start pin: None
---------------------------------------------
 QICK Time Tagger INFO 
---

In [2]:
def sawtooth_1(length=1, maxv=30000, cycles=1):
    data = np.zeros((length*16, 2), dtype=np.int16)
    time = np.linspace(0, 1, length*16) * cycles
    sawtooth = maxv * (time - np.floor(time))
    sawtooth = sawtooth.astype(int)
    data[:,0] = sawtooth
    return data

def sawtooth_2(length=1, maxv=30000, waves=1):
    length     = length*16
    length_wav = int(length/waves)
    data = np.zeros((length, 2), dtype=np.int16)
    ramp = np.linspace(0, maxv, length_wav)
    for ind in range(waves):
        data[:,0][length_wav*ind:length_wav*ind+length_wav] = ramp
    return data


In [24]:
def qick_program (freq):
    reload(qasm)
    freq_int      = soc.freq2reg(freq)
    asm = f"""
// set up pulse parameters
REG_WR w_freq   imm #{freq_int}
REG_WR w_phase  imm #0
REG_WR w_gain   imm #10000
REG_WR w_length imm #32
REG_WR w_env    imm #0

//REG_WR w_conf   imm  #0  // NoPHRST - NoPeriodic - Product
//REG_WR w_conf   imm  #1  // NoPHRST - NoPeriodic - DDS
//REG_WR w_conf   imm  #2  // NoPHRST - NoPeriodic - MEM
//REG_WR w_conf   imm  #4  // NoPHRST - Periodic   - Produc
//REG_WR w_conf   imm  #5  // NoPHRST - Periodic   - DDS
//REG_WR w_conf   imm  #16 // PHRST   - NoPeriodic - Product
//REG_WR w_conf   imm  #17 // PHRST   - NoPeriodic - DDS
//REG_WR w_conf   imm  #18 // PHRST   - NoPeriodic - MEM
//REG_WR w_conf   imm  #20 // PHRST   - Periodic   - Product
//REG_WR w_conf   imm  #21 // PHRST   - Periodic   - DDS

REG_WR out_usr_time imm #50


REG_WR r0 op -op(zero + #1) -uf
LOOP:
    //REG_WR out_usr_time op -op(out_usr_time + #55)
    // play RF pulse
    //REG_WR w_conf   imm  #17 // PHRST   - NoPeriodic - DDS
    //REG_WR w_gain   imm #10000
    //WPORT_WR p0 r_wave
    //WPORT_WR p1 r_wave
    //REG_WR out_usr_time op -op(out_usr_time + #55)
    //TRIG set p0

    //REG_WR out_usr_time op -op(out_usr_time + w_length)
    //TRIG clr p0

   // play RF pulse
    REG_WR w_conf   imm  #18 // PHRST   - NoPeriodic - MEM
    REG_WR w_gain   imm #20000
    WPORT_WR p0 r_wave
    WPORT_WR p1 r_wave
    REG_WR out_usr_time op -op(out_usr_time + #55)
    TRIG set p1

    REG_WR out_usr_time op -op(out_usr_time + w_length)
    TRIG clr p1

    // play RF pulse
    REG_WR w_conf   imm  #16 // PHRST   - NoPeriodic - Product
    REG_WR w_gain   imm #30000
    WPORT_WR p0 r_wave
    WPORT_WR p1 r_wave
    REG_WR out_usr_time op -op(out_usr_time + #55)
    TRIG set p2

    REG_WR out_usr_time op -op(out_usr_time + w_length)
    TRIG clr p2
    REG_WR w_env    op -op(w_env + w_length)
    JUMP LOOP -if(NZ) -wr(r0 op) -op(r0 - #1) -uf

JUMP HERE
    """
    p_txt, p_bin             = Assembler.str_asm2bin(asm)
    soc.tproc.Load_PMEM(p_bin)



In [30]:
freq = 500
data_1 = sawtooth_1(32, 30000, 64)
data_2 = sawtooth_2(32, 20000, 4)

#soc.load_pulse_data(ch, data,addr)
soc.load_pulse_data(0, data_1,0)
soc.load_pulse_data(0, data_2,len(data_1))
soc.load_pulse_data(1, data_1,0)
soc.load_pulse_data(1, data_2,len(data_1))

qick_program(freq)



soc.tproc.start()
time.sleep(.1)
soc.tproc.stop()
